In [1]:
import os 
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import pandas as pd # type: ignore

In [2]:
from utils import get_dataset, get_model_by_tag, get_prompt_template
from LLMAnnotator import LLMAnnotator
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


/home/julita/anaconda3/envs/anote/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
os.environ['OPENAI_API_KEY_CLARIN']=os.getenv("OPENAI_API_KEY_CLARIN")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

token = os.getenv("OPENAI_API_KEY_CLARIN")

In [4]:
import pandas as pd
import re


import pandas as pd

def validate_text_column(path, df_reference, output_col):
    # Wczytaj DataFrame z pliku CSV
    df = pd.read_csv(path)
    
    reference_texts = set(df_reference['text'].unique())

    df['was_in_selected_samples'] = df['text'].isin(reference_texts)
    df.to_csv(path, index=False)
    
    return df

In [5]:

for dataset in ['yahoo', 'sst5']:
    
    print(f'Processing dataset: {dataset}')
    print('')
    print('')
    
    model_tag = 'llama3'  
    prompt = 'emb'
    temp = 0
    number_of_experimet = 1
    selected_samples =  'gaussian'
    
    dataset_tag = dataset
    model = get_model_by_tag(model_tag, token, temp)  
    dataset_for_annotation = get_dataset(dataset_tag=dataset_tag)
    prompt_txt = get_prompt_template(dataset_tag, prompt)
    examples_for_prompt = pd.read_csv(f'./selected_samples/{dataset_tag}/{selected_samples}.csv')

    output_path = f'./results/{prompt}_temp{temp}/{model_tag}/{dataset_tag}_{model_tag}_{prompt}_{selected_samples}_temp{temp}_exp{number_of_experimet}.csv'
    os.makedirs(f'./results/{prompt}_temp{temp}/{model_tag}', exist_ok=True)
    
    
    if not os.path.exists(f'./selected_samples/{dataset}'):
            os.makedirs(f'./selected_samples/{dataset}')
    text_col = "text"
    label_col = "label"
    output_col = "output"


    annotator = LLMAnnotator(
        model=model,
        dataset=dataset_for_annotation,
        examples_for_prompt=examples_for_prompt,
        prompt_template=prompt_txt,
        column_text=text_col,
        column_label=label_col,
        column_output=output_col
    )


    df = annotator.get_results(is_save=True, output_path=output_path)

    annotator.save_results(output_path=output_path)

    df_new = validate_text_column(output_path, examples_for_prompt, output_col)


Processing dataset: yahoo




/home/julita/Desktop/anote-skrpyty 19.04/LLMAnnotator.py:81: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  self.chain = LLMChain(llm=self.model, prompt=self.prompt)


Nr: 0, próba 1/30 – przekroczono timeout 60s, przerywam


Process Process-4:
Traceback (most recent call last):
  File "/home/julita/anaconda3/envs/anote/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/julita/anaconda3/envs/anote/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/julita/Desktop/anote-skrpyty 19.04/LLMAnnotator.py", line 16, in call_chain_process
    result = chain.invoke({"text": text})
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/julita/anaconda3/envs/anote/lib/python3.12/site-packages/langchain/chains/base.py", line 170, in invoke
    raise e
  File "/home/julita/anaconda3/envs/anote/lib/python3.12/site-packages/langchain/chains/base.py", line 160, in invoke
    self._call(inputs, run_manager=run_manager)
  File "/home/julita/anaconda3/envs/anote/lib/python3.12/site-packages/langchain/chains/llm.py", line 126, in _call
    response = self.generate([inputs], run_manager=run_manager)
              

KeyboardInterrupt: 